# FACEBOOK'S PROPHET

## PAGE

- [import](https://www.kaggle.com/znevzz/facebook-s-prophet#Installation)
- [load data](https://www.kaggle.com/znevzz/facebook-s-prophet#DataSets)
- [fit model](https://www.kaggle.com/znevzz/facebook-s-prophet#Default-Model)
- [visualization](https://www.kaggle.com/znevzz/facebook-s-prophet#Let's-visualize)
- [accuracy](https://www.kaggle.com/znevzz/facebook-s-prophet#Accuracy)

## INFO
- [docs](https://facebook.github.io/prophet)
- [github](https://github.com/facebook/prophet/)
- [kaggle example](https://www.kaggle.com/armamut/predicting-transactions-fb-prophet-tutorial)

## Installation
[ISSUE](https://github.com/facebook/prophet/issues/808) and [SOLUTION](https://github.com/facebook/prophet/issues/808#issuecomment-456774908)

In [ ]:
# !pip3 uninstall --yes fbprophet
# !pip3 install fbprophet --no-cache-dir --no-binary :all:

In [ ]:
import logging
logging.getLogger('fbprophet').setLevel(logging.ERROR)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%time

# %matplotlib inline
import pandas as pd
import numpy as np
import fbprophet
from fbprophet.plot import plot_plotly
import plotly.offline as py
import os

py.init_notebook_mode()
fbprophet.__version__

## DataSets

In [ ]:
%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


kaggle_datasets = []
path1 = "../input/fbprophet-sample-1/"
path2 = "../input/the-news-dataset/repository\
/ZNClub-PA-ML-AI-Sentiment-analysis-using-Business-News-82d860a/data/quandl/"
path3 = "../input/timestamp-based/"
for path in (path1, path2, path3):
    kaggle_datasets.extend(map(lambda filename: path + filename, os.listdir(path)))
kaggle_datasets

In [ ]:
%time

def read_timestamp_based(kaggle_datasets):
    df = pd.read_json(kaggle_datasets[-1])
    df['y'] = df['count'].astype('int8')
    df['ds'] = pd.to_datetime(df['time'], format='%Y-%m-%d %H:%M:%S')
    df = df[['ds', 'y']]
    
    return df
# df = pd.read_csv(kaggle_datasets[0]) 
# df.describe()
# df.head()

df = read_timestamp_based(kaggle_datasets)

df = pd.DataFrame(df.groupby('ds').sum().reset_index())
df.describe()

In [ ]:
# partitioned = df[:-364]
partitioned = df[:-20]
partitioned.tail()

## Model

### Default Model

In [ ]:
%time

m = fbprophet.Prophet()
m.fit(partitioned)
# future = m.make_future_dataframe(periods=366, freq='H')
future = m.make_future_dataframe(periods=20, freq='H')
# m.predict(future)
future.tail()


### Using ChangePoints

In [ ]:
%time

# m = fbprophet.Prophet(changepoints=['2012-01-31', '2014-01-31'])
m = fbprophet.Prophet(changepoints=[])
m.fit(partitioned)
# future = m.make_future_dataframe(periods=366)
future = m.make_future_dataframe(periods=21, freq='H')
# m.predict(future)
future.tail()


## Predict

In [ ]:
%time

forecast = m.predict(future)
forecast.tail()

## Let's visualize

In [ ]:
%time

m.plot_components(forecast)

In [ ]:
%time

fig = plot_plotly(m, forecast)  # This returns a plotly Figure
py.iplot(fig)

## Accuracy

In [ ]:
original = df.loc[-365:]
original.tail()

In [ ]:
predicted = forecast[['ds','yhat']]
predicted.tail()

In [ ]:
# Calculate root mean squared error.
diff = forecast.yhat - original.y
np.sqrt(np.mean(diff**2))

# ScratchPad

In [ ]:
# !curl --get https://www.kaggle.com/znevzz

In [ ]:
forecast.head(2)